In [60]:
# Import statements
import pandas as pd
import glob
import matplotlib.pyplot as plt
import os

In [61]:
# Setup OS
FolderPath = os.path.join(os.getcwd(), 'analysis')
# List all files in the 'analysis' folder
files_in_data_folder = os.listdir(FolderPath)
# Print the list of files
print("Files in 'analysis' folder:")
for file in files_in_data_folder:
    print(file)

Files in 'Data' folder:
rmsd_proteinI.xvg
Rg_runI.xvg
rmsf_runII.xvg
Rg_runIII.xvg
rmsf_runIII.xvg
rmsd_proteinIII.xvg
Rg_runII.xvg
rmsd_proteinII.xvg
rmsf_runI.xvg


In [62]:
# Method to create the merged dataframe from a specified data path
def mergeDataFrames(Files, MDCategory, Solvent):
    # Create the dataframe instance
    df_merged = pd.DataFrame()

    # Setup the x axis options (Change accordingly for other analysis)
    x_axis = ''
    if (MDCategory == 'RMSD' or MDCategory == 'Gryate'):
        x_axis = 'Time (ns)'
    elif (MDCategory == 'RMSF' or MDCategory == 'SASA'):
        x_axis = 'Residue'
    elif (MDCategory == 'AtomArea' or MDCategory == 'Area'):
        x_axis = '????'

    # Merge the data from the xvg files into the dataframe
    counter = 0
    for file in Files:
        if (counter == 0):
            df = pd.read_csv(file, delim_whitespace = True, usecols = [0, 1], names=[x_axis , MDCategory + ' Run ' + str(counter + 1)])
            counter+=1
        else:
            df = pd.read_csv(file, delim_whitespace=True, usecols=[1], names=[MDCategory + ' Run ' + str(counter + 1)])
            counter += 1

        if not df_merged.empty:
            df_merged = pd.concat([df_merged, df], axis=1)
        else:
            df_merged = df.copy()

    df_merged.fillna('', inplace=True)

    # Add a column for the average
    if (MDCategory != 'Area'):
        df_merged['Average'] = df_merged.iloc[:, 1:3].mean(numeric_only=True, axis=1)

    return df_merged

In [63]:
# Method to change file names
def ChangeName (files):
    modified_files = []
    for file in files:
        parts = file.split('_')

        for index in range(len(parts)):
            if (parts[index] == 'I'):
                parts[index] = '1'
            elif (parts[index] == 'II'):
                parts[index] = '2'
            elif (parts[index] == 'III'):
                parts[index] = '3'
        file = '_'.join(parts)
        modified_files.append(str(file))
    return modified_files

def ChangeNameReverse (files):
    modified_files = []
    for file in files:
        parts = file.split('_')
        for index in range(len(parts)):
            if (parts[index] == '1'):
                parts[index] = 'I'
            elif (parts[index] == '2'):
                parts[index] = 'II'
            elif (parts[index] == '3'):
                parts[index] = 'III'
        file = '_'.join(parts)
        modified_files.append(str(file))
    return modified_files

# Method to extract data and sort files
def Sort (Data):
    Data = ChangeName(Data)
    Data = sorted(Data)
    Data = ChangeNameReverse(Data)
    return Data


In [64]:
# RMSD Dataframe
RMSD_protein = glob.glob(os.path.join(FolderPath, 'rmsd_protein*.*'))
#RMSD_protein
RMSD_protein = Sort(RMSD_protein)
# Print the list of files for verification
#print("List of RMSD Protein Files:")
#for file_path in RMSD_protein:
    #print(file_path)
df_RMSD_protein = mergeDataFrames(RMSD_protein, 'rmsd', 'Protein')
RMSD_protein

['/content/Data/rmsd_proteinI.xvg',
 '/content/Data/rmsd_proteinII.xvg',
 '/content/Data/rmsd_proteinIII.xvg']

In [65]:
# Plotting RMSD
fig = plt.figure(figsize=(16, 8))

# Check if the DataFrame has at least 5 columns (including the 'Average' column)
if len(df_RMSD_protein.columns) >= 5:
    plt.plot(df_RMSD_protein.iloc[:, 0], df_RMSD_protein.iloc[:, -1], label="Average")

    # Plot individual runs
    for i in range(3):
        plt.plot(df_RMSD_protein.iloc[:, 0], df_RMSD_protein.iloc[:, i + 1], linestyle='--', label=f"Run {i + 1}")

    plt.legend()
    plt.xlabel(df_RMSD_protein.columns[0])
    plt.ylabel('RMSD (nm)')
    plt.title('RMSD Run')
    fig.savefig('Backbone RMSD', dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()
else:
    print("Insufficient columns in DataFrame for plotting RMSD.")


In [66]:
# Plotting RMSD
fig = plt.figure(figsize=(16, 8))
plt.plot(df_RMSD_protein.iloc[:, 0], df_RMSD_protein.iloc[:, 4], label="Average")
for i in range(3):
    plt.plot(df_RMSD_protein.iloc[:, 0], df_RMSD_protein.iloc[:, i + 1], linestyle='--', label=f"Run {i + 1}")
plt.legend()
plt.xlabel(df_RMSD_protein.columns[0])
plt.ylabel('RMSD (nm)')
plt.title('RMSD Run')
fig.savefig('Backbone RMSD', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close()

In [67]:
#Likewise you can work with other analyses in similar fashion...Here are the most common general analyzes RMSF and Radius of Gyration

# Gyrate Dataframe
Gyrate_protein = glob.glob(os.path.join(FolderPath, 'Rg_run*.*'))
Gyrate_protein = Sort(Gyrate_protein)
df_Gyrate_protein = mergeDataFrames(Gyrate_protein, 'Gyrate', 'Protein')

# Plotting Gyrate
fig = plt.figure(figsize=(16, 8))
plt.plot(df_Gyrate_protein.iloc[:, 0], df_Gyrate_protein.iloc[:, 4], label="Average")
for i in range(3):
    plt.plot(df_Gyrate_protein.iloc[:, 0], df_Gyrate_protein.iloc[:, i + 1], linestyle='--', label=f"Run {i + 1}")
plt.legend()
plt.xlabel(df_Gyrate_protein.columns[0])
plt.ylabel('Radius of Gyrate (nm)')
plt.title('Radius of Gyrate Run')
fig.savefig('Gyrate', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close()

# RMSF Backbone Dataframe (Commented Out)
RMSF_protein = glob.glob(os.path.join(FolderPath, 'rmsf_run*.xvg'))
RMSF_protein = Sort(RMSF_protein)
df_RMSF_protein = mergeDataFrames(RMSF_protein, 'RMSF', 'Protein')

# Plotting RMSF
fig = plt.figure(figsize=(16, 8))
plt.plot(df_RMSF_protein.iloc[:, 0], df_RMSF_protein.iloc[:, 4], label="Average")
for i in range(3):
    plt.plot(df_RMSF_protein.iloc[:, 0], df_RMSF_protein.iloc[:, i + 1], linestyle='--', label=f"Run {i + 1}")
plt.legend()
plt.xlabel(df_RMSF_protein.columns[0])
plt.ylabel('RMSF (nm)')
plt.title('RMSF Run')
fig.savefig('Backbone RMSD', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close()

# Export to Excel
writeToExcel = pd.ExcelWriter('Data_protein.xlsx')

df_RMSD_protein.to_excel(writeToExcel, sheet_name='RMSD', index=False)
df_Gyrate_protein.to_excel(writeToExcel, sheet_name='Gyrate', index=False)
df_RMSF_protein.to_excel(writeToExcel, sheet_name='RMSF', index=False)

# Save the Excel file
writeToExcel.save()

<ipython-input-67-0c14c723d1d9>:43: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writeToExcel.save()
